(python-libraries)=
# Useful Python Libraries
The following libraries are available and recommended for use by Cal-ITP data analysts.

## Table of Contents
1. [shared utils](#shared-utils)
1. [calitp](#calitp)
1. [siuba](#siuba)
<br> - [Basic Query](#basic-query)
<br> - [Collect Query Results](#collect-query-results)
<br> - [Show Query SQL](#show-query-sql)
<br> - [More siuba Resources](more-siuba-resources)
1. [pandas](pandas-resources)
1. [Add New Packages](#add-new-packages)

(shared-utils)=
## shared utils
A set of shared utility functions can also be installed, similarly to any Python library. The [shared_utils](https://github.com/cal-itp/data-analyses/shared_utils) are stored here. Generalized functions for analysis are added as collaborative work evolves so we aren't constantly reinventing the wheel.


### In terminal:
* Navigate to the package folder: `cd data-analyses/_shared_utils`
* Use the make command to run through conda install and pip install: `make setup_env`
    * Note: you may need to select Kernel -> Restart Kernel from the top menu after make setup_env in order to successfully import shared_utils
* Alternative: add an `alias` to your `.bash_profile`:
    * In terminal use `cd` to navigate to the home directory (not a repository)
    * Type `nano .bash_profile` to open the .bash_profile in a text editor
    * Add a line at end: `alias go='cd ~/data-analyses/portfolio && pip install -r requirements.txt && cd ../_shared_utils && make setup_env && cd ..'`
    * Exit with Ctrl+X, hit yes, then hit enter at the filename prompt
    * Restart your server; you can check your changes with `cat .bash_profile`


### In notebook:
```python
import shared_utils

#example of using shared_utils
shared_utils.geography_utils.WGS84
```

See [data-analyses/example_reports](https://github.com/cal-itp/data-analyses/tree/main/example_report) for examples on how to use `shared_utils` for general functions, charts, and maps.

(calitp)=
## calitp
`calitp` is an internal library of utility functions used to access our warehouse data.

### import tbls

Most notably, you can include `import tbls` at the top of your notebook to import a table from the warehouse in the form of a `tbls`:

```python
from calitp.tables import tbls
```

Example:

In [1]:
from calitp.tables import tbls

tbls.views.gtfs_schedule_fact_daily_feed_routes()

/opt/hostedtoolcache/Python/3.8.14/x64/lib/python3.8/site-packages/pkg_resources/__init__.py:2456: UserWarning: Obsolete pybigquery is installed, which is likely to
interfere with sqlalchemy_bigquery.
pybigquery should be uninstalled.
  module = __import__(self.module_name, fromlist=['__name__'], level=0)


# Source: lazy query
# DB Conn: Engine(bigquery://cal-itp-data-infra/?maximum_bytes_billed=5000000000)
# Preview:
              feed_key            route_key        date calitp_extracted_at  \
0 -7127830103256469590 -1405500670030456304  2022-03-03          2022-02-22   
1 -8012815880707764251  -144356850092641431  2022-07-07          2022-05-09   
2 -8342008072162760523  5116120563771203630  2022-07-19          2022-07-19   
3 -8348165472045356881 -6258837543168854006  2021-09-16          2021-09-12   
4 -6273133555887087350  5887458509294530339  2022-09-13          2022-09-01   

  calitp_deleted_at  
0        2022-03-15  
1        2022-07-19  
2        2022-07-22  
3        2021-09-26  
4        2022-09-15  
# .. may have more rows

### query_sql

`query_sql` is another useful function to use inside of JupyterHub notebooks to turn a SQL query into a pandas DataFrame.

As an example, in a notebook:

In [2]:
from calitp import query_sql

In [3]:
df_dim_feeds = query_sql("""
SELECT
    *
FROM `views.gtfs_schedule_dim_feeds`
LIMIT 10""", as_df=True)

In [4]:
df_dim_feeds.head()

feed_key  calitp_itp_id  calitp_url_number  \
0 -3971869378563233510              8                  1   
1  4406438169490586349             13                  0   
2 -6443842139930306849             13                  0   
3  3220016414264074104             13                  0   
4 -8012815880707764251             13                  0   

         calitp_agency_name                             raw_gtfs_schedule_url  \
0  Monterey-Salinas Transit      http://www.mst.org/google/google_transit.zip   
1                    Amtrak  https://content.amtrak.com/content/gtfs/GTFS.zip   
2                    Amtrak  https://content.amtrak.com/content/gtfs/GTFS.zip   
3                    Amtrak  https://content.amtrak.com/content/gtfs/GTFS.zip   
4                    Amtrak  https://content.amtrak.com/content/gtfs/GTFS.zip   

   calitp_id_in_latest calitp_feed_id              calitp_feed_name  \
0                False           8__1  Monterey-Salinas Transit (1)   
1                 True          13__0                    Amtrak (0)   
2                 True          13__0                    Amtrak (0)   
3                 True          13__0                    Amtrak (0)   
4                 True          13__0                    Amtrak (0)   

  feed_publisher_name     feed_publisher_url  ... default_lang  \
0                 MST     http://www.mst.org  ...         None   
1              Amtrak  http://www.amtrak.com  ...         None   
2              Amtrak  http://www.amtrak.com  ...         None   
3              Amtrak  http://www.amtrak.com  ...         None   
4              Amtrak  http://www.amtrak.com  ...         None   

             feed_version feed_contact_email feed_contact_url feed_start_date  \
0  202101-February2021v00               None             None            None   
1                    None               None             None      2022-07-09   
2                    None               None             None      2022-03-05   
3                    None               None             None      2022-02-28   
4                    None               None             None      2022-04-30   

  feed_end_date rank  is_composite_feed  calitp_extracted_at calitp_deleted_at  
0          None    1              False           2021-04-15        2021-05-13  
1          None    1              False           2022-07-19        2022-07-25  
2          None    1              False           2022-03-04        2022-05-09  
3          None    1              False           2022-02-28        2022-03-04  
4          None    1              False           2022-05-09        2022-07-19  

[5 rows x 21 columns]

(siuba)=
## siuba
`siuba` is a tool that allows the same analysis code to run on a pandas DataFrame,
as well as generate SQL for different databases.
It supports most [pandas Series methods](https://pandas.pydata.org/pandas-docs/stable/reference/series.html) analysts use. See the [siuba docs](https://siuba.readthedocs.io) for more information.

The examples below go through the basics of using siuba, collecting a database query to a local DataFrame,
and showing SQL test queries that siuba code generates.

### Basic query

In [5]:
from myst_nb import glue
from calitp.tables import tbls
from siuba import _, filter, count, collect, show_query

# query lastest validation notices, then filter for a single gtfs feed,
# and then count how often each code occurs
(tbls.views.gtfs_schedule_dim_feeds()
    >> filter(_.calitp_itp_id == 10, _.calitp_url_number==0)
    >> count(_.feed_key)
)

# Source: lazy query
# DB Conn: Engine(bigquery://cal-itp-data-infra/?maximum_bytes_billed=5000000000)
# Preview:
              feed_key  n
0  2570854701378106641  1
1 -1803822485067769256  1
2   -74403229883010320  1
3 -8003123425407053862  1
4 -8979759065771846304  1
# .. may have more rows

### Collect query results
Note that siuba by default prints out a preview of the SQL query results.
In order to fetch the results of the query as a pandas DataFrame, run `collect()`.

In [6]:
tbl_agency_names = tbls.views.gtfs_schedule_dim_feeds() >> collect()

# Use pandas .head() method to show first 5 rows of data
tbl_agency_names.head()


feed_key  calitp_itp_id  calitp_url_number  \
0 -3971869378563233510              8                  1   
1  4406438169490586349             13                  0   
2 -6443842139930306849             13                  0   
3  3220016414264074104             13                  0   
4 -8012815880707764251             13                  0   

         calitp_agency_name                             raw_gtfs_schedule_url  \
0  Monterey-Salinas Transit      http://www.mst.org/google/google_transit.zip   
1                    Amtrak  https://content.amtrak.com/content/gtfs/GTFS.zip   
2                    Amtrak  https://content.amtrak.com/content/gtfs/GTFS.zip   
3                    Amtrak  https://content.amtrak.com/content/gtfs/GTFS.zip   
4                    Amtrak  https://content.amtrak.com/content/gtfs/GTFS.zip   

   calitp_id_in_latest calitp_feed_id              calitp_feed_name  \
0                False           8__1  Monterey-Salinas Transit (1)   
1                 True          13__0                    Amtrak (0)   
2                 True          13__0                    Amtrak (0)   
3                 True          13__0                    Amtrak (0)   
4                 True          13__0                    Amtrak (0)   

  feed_publisher_name     feed_publisher_url  ... default_lang  \
0                 MST     http://www.mst.org  ...         None   
1              Amtrak  http://www.amtrak.com  ...         None   
2              Amtrak  http://www.amtrak.com  ...         None   
3              Amtrak  http://www.amtrak.com  ...         None   
4              Amtrak  http://www.amtrak.com  ...         None   

             feed_version feed_contact_email feed_contact_url feed_start_date  \
0  202101-February2021v00               None             None            None   
1                    None               None             None      2022-07-09   
2                    None               None             None      2022-03-05   
3                    None               None             None      2022-02-28   
4                    None               None             None      2022-04-30   

  feed_end_date rank  is_composite_feed  calitp_extracted_at calitp_deleted_at  
0          None  1.0              False           2021-04-15        2021-05-13  
1          None  1.0              False           2022-07-19        2022-07-25  
2          None  1.0              False           2022-03-04        2022-05-09  
3          None  1.0              False           2022-02-28        2022-03-04  
4          None  1.0              False           2022-05-09        2022-07-19  

[5 rows x 21 columns]

### Show query SQL

While `collect()` fetches query results, `show_query()` prints out the SQL code that siuba generates.

In [7]:
(tbls.views.gtfs_schedule_dim_feeds()
  >> filter(_.calitp_agency_name.str.contains("Metro"))
  >> show_query(simplify=True)
)


SELECT `anon_1`.`feed_key`, `anon_1`.`calitp_itp_id`, `anon_1`.`calitp_url_number`, `anon_1`.`calitp_agency_name`, `anon_1`.`raw_gtfs_schedule_url`, `anon_1`.`calitp_id_in_latest`, `anon_1`.`calitp_feed_id`, `anon_1`.`calitp_feed_name`, `anon_1`.`feed_publisher_name`, `anon_1`.`feed_publisher_url`, `anon_1`.`feed_lang`, `anon_1`.`default_lang`, `anon_1`.`feed_version`, `anon_1`.`feed_contact_email`, `anon_1`.`feed_contact_url`, `anon_1`.`feed_start_date`, `anon_1`.`feed_end_date`, `anon_1`.`rank`, `anon_1`.`is_composite_feed`, `anon_1`.`calitp_extracted_at`, `anon_1`.`calitp_deleted_at` 
FROM (SELECT feed_key, calitp_itp_id, calitp_url_number, calitp_agency_name, raw_gtfs_schedule_url, calitp_id_in_latest, calitp_feed_id, calitp_feed_name, feed_publisher_name, feed_publisher_url, feed_lang, default_lang, feed_version, feed_contact_email, feed_contact_url, feed_start_date, feed_end_date, rank, is_composite_feed, calitp_extracted_at, calitp_deleted_at 
FROM `views.gtfs_schedule_dim_feeds

# Source: lazy query
# DB Conn: Engine(bigquery://cal-itp-data-infra/?maximum_bytes_billed=5000000000)
# Preview:
              feed_key  calitp_itp_id  calitp_url_number  \
0 -5156377554606803196            293                  0   
1 -3897638344784195057            296                  0   
2 -4045042305146455543            296                  0   
3 -6941377690184990428            296                  0   
4  6961970883549490697            323                  0   

                            calitp_agency_name  \
0  Santa Barbara Metropolitan Transit District   
1     Santa Cruz Metropolitan Transit District   
2     Santa Cruz Metropolitan Transit District   
3     Santa Cruz Metropolitan Transit District   
4                                    Metrolink   

                               raw_gtfs_schedule_url  calitp_id_in_latest  \
0           http://sbmtd.gov/google_transit/feed.zip                 True   
1  https://scmtd.com/google_transit/google_transi...                 True   
2  http://scmtd.com/google_transit/google_transit...                 True   
3  http://scmtd.com/google_transit/google_transit...                 True   
4  https://www.metrolinktrains.com/globalassets/a...                 True   

  calitp_feed_id                                 calitp_feed_name  \
0         293__0  Santa Barbara Metropolitan Transit District (0)   
1         296__0     Santa Cruz Metropolitan Transit District (0)   
2         296__0     Santa Cruz Metropolitan Transit District (0)   
3         296__0     Santa Cruz Metropolitan Transit District (0)   
4         323__0                                    Metrolink (0)   

  feed_publisher_name              feed_publisher_url  ... default_lang  \
0   Santa Barbara MTD            http://www.sbmtd.gov  ...         None   
1    Santa Cruz Metro            http://www.scmtd.com  ...         None   
2    Santa Cruz Metro            http://www.scmtd.com  ...         None   
3    Santa Cruz Metro            http://www.scmtd.com  ...         None   
4    Metrolink Trains  http://www.metrolinktrains.com  ...         None   

               feed_version     feed_contact_email  \
0  220425 April 25_20220331                   None   
1                      None  gtfsupdates@scmtd.com   
2                      None  gtfsupdates@scmtd.com   
3                      None  gtfsupdates@scmtd.com   
4                  20210106                   None   

                                    feed_contact_url feed_start_date  \
0                                               None      2022-04-25   
1  https://www.scmtd.com/en/riders-guide/about-go...            None   
2  https://scmtd.com/en/riders-guide/google-trans...            None   
3  https://www.scmtd.com/en/riders-guide/about-go...            None   
4                                               None            None   

  feed_end_date rank  is_composite_feed  calitp_extracted_at calitp_deleted_at  
0    2022-08-14    1              False           2022-04-04        2022-04-18  
1          None    1              False           2022-10-25        2099-01-01  
2          None    1              False           2021-09-07        2021-12-10  
3          None    1              False           2021-12-10        2022-10-25  
4          None    1              False           2021-04-15        2021-05-24  

[5 rows x 21 columns]
# .. may have more rows

Note that here the pandas Series method `str.contains` corresponds to `regexp_contains` in Google BigQuery.

(more-siuba-resources)=
### More siuba Resources:
* [siuba docs](https://siuba.readthedocs.io)
* ['Tidy Tuesday' live analyses with siuba](https://www.youtube.com/playlist?list=PLiQdjX20rXMHc43KqsdIowHI3ouFnP_Sf)


(pandas-resources)=
## pandas
The library pandas is very commonly used in data analysis, and the external resources below provide a brief overview of it's use.

* [Cheat Sheet - pandas](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)

## Add New Packages

While most Python packages an analyst uses come in JupyterHub, there may be additional packages you'll want to use in your analysis.

* Install [shared utility functions](#shared-utils)
* Change directory into the project task's subfolder and add `requirements.txt` and/or `conda-requirements.txt`
* Run `pip install -r requirements.txt` and/or `conda install --yes -c conda-forge --file conda-requirements.txt`